In [7]:
!python -m pip install pysimplegui
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import PySimpleGUI as sg
import sys

sys.path.append("../")
from src.alpha_connect.connect4_game import Connect4Game
from src.alpha_connect.connect4_supervised_agent import Connect4SupervisedAgent
from src.alpha_connect.connect4_random_agent import Connect4RandomAgent
from src.alpha_connect.connect4_MCTS_agent import Connect4MCTSAgent
from src.alpha_connect.connect4_human_agent import Connect4HumanAgent


def create_connect4_board(board_rows, board_columns):
    """Create the Connect 4 game board layout using circles."""
    cell_size = 60  # Size of each cell in the grid
    circle_radius = 25  # Radius of each circle

    # Calculate the size of the graph canvas
    graph_canvas_size = (board_columns * cell_size, board_rows * cell_size)

    players = {
        "Human": Connect4HumanAgent(),
        "Supervised": Connect4SupervisedAgent("epoch=72-step=18688.ckpt"),
        "Random": Connect4RandomAgent(),
        "MCTS": Connect4MCTSAgent(),
    }

    # Create a Graph element to draw the circles
    graph = sg.Graph(
        canvas_size=graph_canvas_size,
        graph_bottom_left=(0, 0),
        graph_top_right=graph_canvas_size,
        background_color="blue",
        key="-GRAPH-",
        enable_events=True,
    )

    # Define the layout for the menu on the right
    menu_layout = [
        [sg.Button("Reset", key="-RESET-")],
        [sg.Button("Play", key="-PLAY-")],
        [sg.Text("Player 1:")],
        [sg.DropDown(["Human", "Supervised", "Random", "MCTS"], key="-PLAYER1-")],
        [sg.Text("Player 2:")],
        [sg.DropDown(["Human", "Supervised", "Random", "MCTS"], key="-PLAYER2-")],
    ]

    # Combine the graph and menu layouts
    layout = [[graph, sg.Column(menu_layout)]]

    # Create the window
    window = sg.Window("Connect 4", layout, finalize=True)

    # Draw the initial board with empty circles (white)
    for row in range(board_rows):
        for col in range(board_columns):
            center_x = col * cell_size + cell_size // 2
            center_y = row * cell_size + cell_size // 2
            graph.DrawCircle(
                (center_x, center_y),
                circle_radius,
                fill_color="white",
                line_color="white",
            )

    game = None
    # Event loop
    while True:
        event, values = window.read()
        if event == sg.WIN_CLOSED:
            break
        elif event == "-GRAPH-":  # If a graph area is clicked
            if game is None:
                continue
            x, y = values["-GRAPH-"]
            col = x // cell_size
            Connect4HumanAgent.add_waiting_move(col)

            for row in range(board_rows):
                for col in range(board_columns):
                    center_x = col * cell_size + cell_size // 2
                    center_y = row * cell_size + cell_size // 2
                    graph.DrawCircle(
                        (center_x, center_y),
                        circle_radius,
                        fill_color="white",
                        line_color="white",
                    )

        elif event == "-RESET-":
            game = None

            for row in range(board_rows):
                for col in range(board_columns):
                    center_x = col * cell_size + cell_size // 2
                    center_y = row * cell_size + cell_size // 2
                    graph.DrawCircle(
                        (center_x, center_y),
                        circle_radius,
                        fill_color="white",
                        line_color="white",
                    )
            print("Reset the game logic here")

        elif event == "-PLAY-":
            agent1 = players.get(values["-PLAYER1-"], Connect4HumanAgent())
            agent2 = players.get(values["-PLAYER2-"], Connect4HumanAgent())
            print("Play button logic here")
            if not play_game(
                agent1,
                agent2,
                graph,
                cell_size,
                circle_radius,
                board_rows,
                board_columns,
                window,
            ):
                break  # Exit the game if the window is closed

    window.close()


def update_board(graph, cell_size, circle_radius, board_rows, board_columns, game):
    colors = {-1: "white", 0: "yellow", 1: "red"}
    for row in range(board_rows):
        for col in range(board_columns):
            center_x = col * cell_size + cell_size // 2
            center_y = row * cell_size + cell_size // 2
            color = colors[game.get_value(col, row)]
            graph.DrawCircle(
                (center_x, center_y), circle_radius, fill_color=color, line_color=color
            )


def play_game(
    agent1, agent2, graph, cell_size, circle_radius, board_rows, board_columns, window
):
    game = Connect4Game(agent1, agent2)
    agents = [agent1, agent2]
    current_player = 0
    while not game.has_ended():
        window.refresh()
        if not isinstance(agents[current_player], Connect4HumanAgent):
            print("Playing move")
            game = game.play()
            current_player = 1 - current_player
            update_board(
                graph, cell_size, circle_radius, board_rows, board_columns, game
            )
            continue

        event, values = window.read()
        if event == sg.WIN_CLOSED:
            return False
        elif event == "-RESET-":
            game = game.reset()
            update_board(
                graph, cell_size, circle_radius, board_rows, board_columns, game
            )
            return True
        elif event == "-GRAPH-":  # If a graph area is clicked
            if game is None:
                continue
            x, y = values["-GRAPH-"]
            col = x // cell_size
            Connect4HumanAgent.add_waiting_move(col)
            game = game.play()
            current_player = 1 - current_player

            update_board(
                graph, cell_size, circle_radius, board_rows, board_columns, game
            )

    update_board(graph, cell_size, circle_radius, board_rows, board_columns, game)

    return True


if __name__ == "__main__":
    # Constants for the board size
    BOARD_ROWS = 6
    BOARD_COLUMNS = 7
    create_connect4_board(BOARD_ROWS, BOARD_COLUMNS)

Play button logic here
Playing move
tensor([[-3.3900,  1.4151, -0.7727,  1.0921,  0.3966,  0.1480, -0.8059]],
       device='mps:0', grad_fn=<LinearBackward0>) tensor([[0.6776]], device='mps:0', grad_fn=<TanhBackward0>)


/Users/alberttroussard/Documents/connect4project/connect4_supervised_agent.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_hat,value = self.model(torch.tensor(input, device='mps').type(torch.float32).view(1,3,6,7))


Playing move
tensor([[-1.8944, -0.4326, -1.1811,  1.1263,  1.3570, -1.5859, -1.9001]],
       device='mps:0', grad_fn=<LinearBackward0>) tensor([[0.5452]], device='mps:0', grad_fn=<TanhBackward0>)


/Users/alberttroussard/Documents/connect4project/connect4_supervised_agent.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_hat,value = self.model(torch.tensor(input, device='mps').type(torch.float32).view(1,3,6,7))


Playing move
tensor([[-3.7454, -0.5442, -1.5982,  6.6076, -0.9766, -1.5362, -2.2517]],
       device='mps:0', grad_fn=<LinearBackward0>) tensor([[0.2788]], device='mps:0', grad_fn=<TanhBackward0>)


...

Playing move
tensor([[-2.8906, -0.7439,  2.0297,  2.3487, -0.4608, -0.4669, -2.4620]],
       device='mps:0', grad_fn=<LinearBackward0>) tensor([[0.1316]], device='mps:0', grad_fn=<TanhBackward0>)


/Users/alberttroussard/Documents/connect4project/connect4_supervised_agent.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_hat,value = self.model(torch.tensor(input, device='mps').type(torch.float32).view(1,3,6,7))


Playing move
tensor([[ 2.1174, -2.1386, -0.1550, -2.0652, -0.8072, -0.9052, -3.0463]],
       device='mps:0', grad_fn=<LinearBackward0>) tensor([[0.0941]], device='mps:0', grad_fn=<TanhBackward0>)


/Users/alberttroussard/Documents/connect4project/connect4_supervised_agent.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_hat,value = self.model(torch.tensor(input, device='mps').type(torch.float32).view(1,3,6,7))


Reset the game logic here
